In [1]:
# Import libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import glob
import numpy as np

print("✅ Libraries loaded")

✅ Libraries loaded


## 1. Load and Combine Data

In [2]:
# Load all team cap CSV files
data_dir = '../data/raw'
files = glob.glob(f'{data_dir}/spotrac_team_cap_*_*.csv')

# Get the most recent file for each year
year_files = {}
for f in files:
    parts = Path(f).stem.split('_')
    year = int(parts[3])
    if year not in year_files or f > year_files[year]:
        year_files[year] = f

# Load and combine
dfs = []
for year, filepath in sorted(year_files.items()):
    df = pd.read_csv(filepath)
    df['year'] = year
    dfs.append(df)
    print(f"✓ {year}: {len(df)} teams, ${df['dead_money_millions'].sum():.1f}M total")

df = pd.concat(dfs, ignore_index=True)
print(f"\n✅ Combined: {len(df)} team-year records ({df['year'].min()}-{df['year'].max()})")
print(f"   Teams: {df['team'].nunique()} unique")
print(f"   Years: {df['year'].nunique()} years")

✓ 2015: 32 teams, $551.7M total
✓ 2016: 32 teams, $496.4M total
✓ 2017: 32 teams, $501.5M total
✓ 2018: 32 teams, $672.7M total
✓ 2019: 32 teams, $771.5M total
✓ 2020: 32 teams, $797.6M total
✓ 2021: 32 teams, $917.6M total
✓ 2022: 32 teams, $1154.5M total
✓ 2023: 32 teams, $1236.0M total
✓ 2024: 32 teams, $1559.0M total

✅ Combined: 320 team-year records (2015-2024)
   Teams: 35 unique
   Years: 10 years


In [3]:
# Preview data
df.head(10)

,team,year,active_cap_millions,dead_money_millions,total_cap_millions,cap_space_millions,dead_cap_pct
0,JAX,2015,107.546334,9.582465,136.130404,32.355703,7.04
1,TEN,2015,90.472606,17.866172,128.662671,25.690907,13.89
2,CLE,2015,102.057621,7.054744,139.647560,22.119840,5.05
3,NYG,2015,78.694040,15.599806,128.321422,15.090461,12.16
4,OAK,2015,94.352516,16.688813,137.071838,14.800918,12.18
5,SF,2015,74.501722,29.094100,136.740459,13.115521,21.28
6,TB,2015,86.632966,25.305733,133.567282,11.342607,18.95
7,DAL,2015,80.128732,19.218914,137.816498,10.761815,13.95
8,ATL,2015,99.720594,22.469541,136.236065,10.012750,16.49
9,GB,2015,125.157906,5.436397,141.856037,9.615069,3.83


## 2. Data Quality Checks

In [4]:
# Check for missing data
print("Missing values:")
print(df.isnull().sum())

print("\nData types:")
print(df.dtypes)

print("\nBasic statistics:")
print(df[['dead_money_millions', 'total_cap_millions', 'dead_cap_pct']].describe())

Missing values:
team                   0
year                   0
active_cap_millions    0
dead_money_millions    0
total_cap_millions     0
cap_space_millions     0
dead_cap_pct           0
dtype: int64

Data types:
team                    object
year                     int64
active_cap_millions    float64
dead_money_millions    float64
total_cap_millions     float64
cap_space_millions     float64
dead_cap_pct           float64
dtype: object

Basic statistics:
       dead_money_millions  total_cap_millions  dead_cap_pct
count           320.000000          320.000000    320.000000
mean             27.057710          188.533967     13.928438
std              18.231487           32.337787      8.145011
min               3.235487          128.321422      2.120000
25%              14.102820          161.689951      8.140000
50%              22.254284          187.071640     12.270000
75%              33.383111          210.010053     16.940000
max              93.289973          261.52511

## 3. Summary Statistics

In [5]:
# Yearly summary
yearly_summary = df.groupby('year').agg({
    'dead_money_millions': ['sum', 'mean', 'median', 'std', 'min', 'max'],
    'dead_cap_pct': ['mean', 'median']
})
yearly_summary.columns = ['_'.join(col) for col in yearly_summary.columns]
print("Dead Money by Year:")
print(yearly_summary.round(1))

Dead Money by Year:
      dead_money_millions_sum  dead_money_millions_mean  \
year                                                      
2015                    551.7                      17.2   
2016                    496.4                      15.5   
2017                    501.5                      15.7   
2018                    672.7                      21.0   
2019                    771.5                      24.1   
2020                    797.6                      24.9   
2021                    917.6                      28.7   
2022                   1154.5                      36.1   
2023                   1236.0                      38.6   
2024                   1559.0                      48.7   

      dead_money_millions_median  dead_money_millions_std  \
year                                                        
2015                        17.2                      7.6   
2016                        14.7                      8.5   
2017                       

In [6]:
# Team summary (total dead money 2015-2024)
team_summary = df.groupby('team').agg({
    'dead_money_millions': ['sum', 'mean', 'max'],
    'dead_cap_pct': 'mean'
}).round(1)
team_summary.columns = ['_'.join(col) for col in team_summary.columns]
team_summary = team_summary.sort_values('dead_money_millions_sum', ascending=False)
print("\nTop 10 Teams by Total Dead Money (2015-2024):")
print(team_summary.head(10))


Top 10 Teams by Total Dead Money (2015-2024):
      dead_money_millions_sum  dead_money_millions_mean  \
team                                                      
PHI                     379.2                      37.9   
NYG                     373.1                      37.3   
CAR                     363.8                      36.4   
NO                      360.4                      36.0   
HOU                     321.2                      32.1   
MIA                     315.5                      31.5   
DEN                     305.2                      30.5   
DET                     299.1                      29.9   
CLE                     295.5                      29.5   
NYJ                     294.1                      29.4   

      dead_money_millions_max  dead_cap_pct_mean  
team                                              
PHI                      64.5               18.9  
NYG                      82.5               18.7  
CAR                      73.2           

## 4. Stacked Bar Chart: Dead Money by Team and Year

In [7]:
# NFL team colors
TEAM_COLORS = {
    'ARI': '#97233F', 'ATL': '#A71930', 'BAL': '#241773', 'BUF': '#00338D',
    'CAR': '#0085CA', 'CHI': '#0B162A', 'CIN': '#FB4F14', 'CLE': '#311D00',
    'DAL': '#041E42', 'DEN': '#FB4F14', 'DET': '#0076B6', 'GB': '#203731',
    'HOU': '#03202F', 'IND': '#002C5F', 'JAX': '#006778', 'KC': '#E31837',
    'LA': '#003594', 'LAC': '#0080C6', 'LAR': '#003594', 'LV': '#000000',
    'MIA': '#008E97', 'MIN': '#4F2683', 'NE': '#002244', 'NO': '#D3BC8D',
    'NYG': '#0B2265', 'NYJ': '#125740', 'OAK': '#000000', 'PHI': '#004C54',
    'PIT': '#FFB612', 'SD': '#0080C6', 'SEA': '#002244', 'SF': '#AA0000',
    'STL': '#003594', 'TB': '#D50A0A', 'TEN': '#0C2340', 'WAS': '#5A1414',
}

# Pivot data
pivot = df.pivot(index='year', columns='team', values='dead_money_millions').fillna(0)

# Sort teams by total dead money
team_totals = pivot.sum().sort_values(ascending=False)
pivot = pivot[team_totals.index]

print(f"Creating stacked bar chart with {len(pivot.columns)} teams...")

Creating stacked bar chart with 35 teams...


In [8]:
# Create stacked bar chart
fig = go.Figure()

for team in pivot.columns:
    color = TEAM_COLORS.get(team, '#888888')
    fig.add_trace(go.Bar(
        name=team,
        x=pivot.index,
        y=pivot[team],
        marker_color=color,
        hovertemplate=f'<b>{team}</b><br>Year: %{{x}}<br>Dead Money: $%{{y:.1f}}M<extra></extra>'
    ))

# Calculate totals for annotations
totals = pivot.sum(axis=1)

fig.update_layout(
    title={
        'text': 'NFL Dead Money by Team (2015-2024)<br><sub>Stacked by Team</sub>',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Year',
    yaxis_title='Dead Money (Millions USD)',
    barmode='stack',
    hovermode='closest',
    template='plotly_white',
    height=700,
    width=1200,
    showlegend=True,
    legend=dict(
        orientation='v',
        yanchor='top',
        y=1,
        xanchor='left',
        x=1.02,
        font=dict(size=9)
    ),
    xaxis=dict(tickmode='linear', tick0=pivot.index.min(), dtick=1),
    yaxis=dict(tickprefix='$', ticksuffix='M')
)

# Add total annotations
for year in pivot.index:
    fig.add_annotation(
        x=year, y=totals[year],
        text=f'${totals[year]:.0f}M',
        showarrow=False, yshift=10,
        font=dict(size=10, color='black')
    )

fig.show()

# Save
output_path = Path('../notebooks/outputs/dead_money_stacked_by_team.html')
output_path.parent.mkdir(parents=True, exist_ok=True)
fig.write_html(str(output_path))
print(f"\n✅ Saved to: {output_path}")


✅ Saved to: ../notebooks/outputs/dead_money_stacked_by_team.html


## 5. Trend Analysis: Total Dead Money Over Time

In [9]:
# Aggregate by year
yearly = df.groupby('year').agg({
    'dead_money_millions': ['sum', 'mean', 'std']
}).reset_index()
yearly.columns = ['year', 'total_dead_money', 'avg_per_team', 'std_dev']

# Create dual-axis chart
fig = go.Figure()

# Total dead money (bar)
fig.add_trace(go.Bar(
    name='Total League Dead Money',
    x=yearly['year'],
    y=yearly['total_dead_money'],
    marker_color='#AA0000',
    yaxis='y',
    hovertemplate='Year: %{x}<br>Total: $%{y:.1f}M<extra></extra>'
))

# Average per team (line)
fig.add_trace(go.Scatter(
    name='Avg Per Team',
    x=yearly['year'],
    y=yearly['avg_per_team'],
    mode='lines+markers',
    line=dict(color='#0080C6', width=3),
    marker=dict(size=8),
    yaxis='y2',
    hovertemplate='Year: %{x}<br>Avg: $%{y:.1f}M per team<extra></extra>'
))

fig.update_layout(
    title={
        'text': 'NFL Dead Money Trends (2015-2024)<br><sub>Total League vs Average Per Team</sub>',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Year',
    template='plotly_white',
    height=600,
    width=1000,
    hovermode='x unified',
    yaxis=dict(
        title='Total League Dead Money (Millions USD)',
        tickprefix='$', ticksuffix='M', side='left'
    ),
    yaxis2=dict(
        title='Average Per Team (Millions USD)',
        tickprefix='$', ticksuffix='M',
        overlaying='y', side='right'
    ),
    xaxis=dict(tickmode='linear', tick0=yearly['year'].min(), dtick=1)
)

fig.show()

# Save
output_path = Path('../notebooks/outputs/dead_money_yearly_trend.html')
fig.write_html(str(output_path))
print(f"\n✅ Saved to: {output_path}")


✅ Saved to: ../notebooks/outputs/dead_money_yearly_trend.html


## 6. Key Insights

In [10]:
# Calculate growth rate
start_year = yearly['year'].min()
end_year = yearly['year'].max()
start_total = yearly[yearly['year'] == start_year]['total_dead_money'].values[0]
end_total = yearly[yearly['year'] == end_year]['total_dead_money'].values[0]
growth_pct = ((end_total - start_total) / start_total) * 100

print(f"\n{'='*60}")
print("KEY INSIGHTS")
print(f"{'='*60}\n")

print(f"1. Total dead money grew {growth_pct:+.1f}% from {start_year} to {end_year}")
print(f"   - {start_year}: ${start_total:.1f}M")
print(f"   - {end_year}: ${end_total:.1f}M")

print(f"\n2. Peak year: {yearly.loc[yearly['total_dead_money'].idxmax(), 'year']:.0f}")
print(f"   - Total: ${yearly['total_dead_money'].max():.1f}M")

print(f"\n3. Top 3 teams by total dead money ({start_year}-{end_year}):")
for i, (team, total) in enumerate(team_summary.head(3)['dead_money_millions_sum'].items(), 1):
    print(f"   {i}. {team}: ${total:.1f}M")

print(f"\n4. Average dead cap percentage: {df['dead_cap_pct'].mean():.1f}%")
print(f"   - Range: {df['dead_cap_pct'].min():.1f}% - {df['dead_cap_pct'].max():.1f}%")


KEY INSIGHTS

1. Total dead money grew +182.6% from 2015 to 2024
   - 2015: $551.7M
   - 2024: $1559.0M

2. Peak year: 2024
   - Total: $1559.0M

3. Top 3 teams by total dead money (2015-2024):
   1. PHI: $379.2M
   2. NYG: $373.1M
   3. CAR: $363.8M

4. Average dead cap percentage: 13.9%
   - Range: 2.1% - 45.6%
